# **Colab**

In [1]:
# === 1. Montagem do Google Drive e definição dos caminhos ===
from google.colab import drive
drive.mount('/content/drive')
caminho_base = '/content/drive/My Drive/Colab/cb-classificador/data/'
arquivo_tratado     = caminho_base + 'dados_tratados.csv'
arquivo_codificado  = caminho_base + 'dados_codificados.csv'
arquivo_vocabulario = caminho_base + 'dados_vocabulario.csv'
arquivo_encoder     = caminho_base + 'onehot_encoder.pkl'
arquivo_vectorizer  = caminho_base + 'tfidf_vectorizer.pkl'

Mounted at /content/drive


In [2]:
# === 2. Importação de bibliotecas ===
import pandas as pd
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# === 3. Função para codificar variáveis categóricas ===
def codificar_variaveis_categoricas(df, colunas_categoricas, caminho_encoder):
    encoder = OneHotEncoder(handle_unknown='ignore')
    X_categorico = encoder.fit_transform(df[colunas_categoricas])
    joblib.dump(encoder, caminho_encoder) # Armazenar o encoder para uso no protótipo
    return X_categorico, encoder

In [4]:
# === 4. Função para vetorizar variável textual ===
def vetorizar_texto(df, coluna_texto, caminho_vectorizer, caminho_vocabulario, min_df=10):
    vectorizer = TfidfVectorizer(min_df=min_df)
    X_texto = vectorizer.fit_transform(df[coluna_texto])
    joblib.dump(vectorizer, caminho_vectorizer) # Armazenar o vectorizer para uso no protótipo
    df_vocab = pd.DataFrame(vectorizer.get_feature_names_out(), columns=['Termo'])
    df_vocab.to_csv(caminho_vocabulario, index=False) # Armazenar o vocabulário
    return X_texto

In [5]:
# === 5. Função para codificar, vetorizar e preparar dados ===
def codificar_vetorizar_dados(df, colunas_categoricas, coluna_texto, caminho_encoder, caminho_vectorizer, caminho_vocabulario):
    X_categorico, encoder = codificar_variaveis_categoricas(df, colunas_categoricas, caminho_encoder)
    df[coluna_texto] = df['des_anp'].fillna('') + ' | ' + df['des_produto'].fillna('')
    X_texto = vetorizar_texto(df, coluna_texto, caminho_vectorizer, caminho_vocabulario)
    X = pd.concat([
        pd.DataFrame(X_categorico.toarray(), columns=encoder.get_feature_names_out(colunas_categoricas)),
        pd.DataFrame(X_texto.toarray())
    ], axis=1)
    X.columns = X.columns.astype(str)
    y = df['categoria']
    return X, y

In [6]:
# === 6. Leitura dos dados ===
df = pd.read_csv(arquivo_tratado)

In [7]:
# === 7. Verificação do balanceamento das classes da variável alvo ===
total = df['categoria'].count()
print(f"Percentual CB: {df['categoria'].value_counts().get('CB', 0)/total*100:.2f}%")
print(f"Percentual XX: {df['categoria'].value_counts().get('XX', 0)/total*100:.2f}%")

Percentual CB: 42.01%
Percentual XX: 57.99%


In [8]:
len(df)

1914

In [9]:
df['categoria'].value_counts()

,count
categoria,
XX,1110
CB,804


In [10]:
# === 8. Aplicação do pipeline de codificação e vetorização ===
X, y = codificar_vetorizar_dados(
    df=df,
    colunas_categoricas=['cod_ncm', 'cod_anp', 'cod_unidade'],
    coluna_texto='txt_descricao',
    caminho_encoder=arquivo_encoder,
    caminho_vectorizer=arquivo_vectorizer,
    caminho_vocabulario=arquivo_vocabulario
)

In [11]:
# === 9. Armazenar arquivo codificado ===
df_codificado = pd.concat([X, y], axis=1)
df_codificado.to_csv(arquivo_codificado, index=False)

In [12]:
len(df_codificado)

1914

In [13]:
len(df_codificado.columns)

186